# 🧪 AI-Guided SIR Modeling Hackathon (2 Hours)

This notebook is the **workshop skeleton** for the **AI-Guided SIR Modeling Hackathon**.

- Dataset: **Niamey, Niger measles outbreaks** (biweekly case counts; communities A/B/C)
- Tooling: **Google Colab + Python + ChatGPT**
- Time unit: **biweeks**
- Key modeling shortcut: infectious period ≈ **2 weeks** ⇒ **γ ≈ 1 per biweek**

---

## How to use this notebook
Each section contains:
1) A **ChatGPT prompt** (Markdown) you can copy into ChatGPT  
2) A **starter code cell** (often with TODOs) to run in Colab

✅ **Rule of thumb:** Always request **plots + sanity checks** when you ask ChatGPT for help.


## 0. Setup

### ChatGPT Prompt (copy/paste)
```text
Persona:
You are an expert epidemiological modeler and Python educator.

Context:
- Dataset: biweekly measles case counts from Niamey, Niger
- Columns: biweek, community, measles
- Data represent: observed case counts (incidence)
- Time unit: biweeks
- Infectious period ≈ 2 weeks (gamma ≈ 1 per biweek)

Task:
Write a Colab-ready setup cell: imports, basic plotting config, and a helper for reproducibility.

Constraints:
- Use numpy, scipy, pandas, matplotlib only

Verification:
- Print library versions and a simple "ready" message.
```


In [ ]:
# Imports (Colab-friendly)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.integrate import solve_ivp
from scipy.optimize import minimize, least_squares

# Reproducibility
np.random.seed(42)

print("Ready ✅")
print("numpy:", np.__version__)
print("pandas:", pd.__version__)


## 1. Load & Explore the Niamey Measles Data

### ChatGPT Prompt (copy/paste)
```text
Persona:
You are an expert epidemiological modeler and Python educator.

Context:
- Dataset contains biweekly measles case counts from Niamey, Niger
- Columns: biweek, community, measles

Task:
Write Python code to:
1) Load data/niamey.csv
2) Plot measles cases over time for each community (A, B, C)
3) Briefly explain what biweekly case counts mean for modeling

Constraints:
- Use pandas + matplotlib only

Verification:
- One time-series plot with labeled axes and legend
```


In [ ]:
# Load dataset
# In this repo: data/niamey.csv
# If running from the provided environment, you may also have /mnt/data/niamey.csv available.
import os

candidate_paths = [
    "data/niamey.csv",
    "/mnt/data/niamey.csv",
]

data_path = next((p for p in candidate_paths if os.path.exists(p)), None)
if data_path is None:
    raise FileNotFoundError("Could not find niamey.csv. Expected at data/niamey.csv (repo) or /mnt/data/niamey.csv (environment).")

df = pd.read_csv(data_path)
display(df.head())
print("Rows:", len(df))
print("Columns:", list(df.columns))

# Basic plot: measles by community over biweek
plt.figure()
for comm, g in df.groupby("community"):
    plt.plot(g["biweek"], g["measles"], marker="o", label=str(comm))
plt.xlabel("Biweek")
plt.ylabel("Measles case counts")
plt.title("Niamey measles outbreaks (biweekly counts) by community")
plt.legend()
plt.grid(True)
plt.show()


## 2. Implement the SIR Model (Biweekly Units)

We use a **closed SIR** model for a single outbreak wave:
- S(t): susceptible
- I(t): infectious
- R(t): recovered

Time unit = **biweeks**.

### ChatGPT Prompt (copy/paste)
```text
Persona:
You are an expert epidemiological modeler and Python educator.

Context:
- Closed SIR model
- Time unit: biweeks
- Infectious period ≈ 2 weeks (gamma ≈ 1 per biweek)

Task:
Implement an SIR model using SciPy ODE solving:
- Define the SIR equations
- Solve over a biweekly time grid
- Plot S, I, R over time

Constraints:
- Use scipy.integrate.solve_ivp
- Keep code readable and commented

Verification:
- Plot S, I, R
- Verify S + I + R ≈ N
```


In [ ]:
def sir_rhs(t, y, beta, gamma, N):
    """Closed SIR model RHS.
    y = [S, I, R]
    t in biweeks
    beta: transmission rate per biweek
    gamma: recovery rate per biweek (~1 for measles with 2-week infectious period)
    """
    S, I, R = y
    dS = -beta * S * I / N
    dI = beta * S * I / N - gamma * I
    dR = gamma * I
    return [dS, dI, dR]

def simulate_sir(t_eval, beta, gamma, N, I0, R0=0.0):
    S0 = N - I0 - R0
    sol = solve_ivp(
        fun=lambda t, y: sir_rhs(t, y, beta=beta, gamma=gamma, N=N),
        t_span=(float(np.min(t_eval)), float(np.max(t_eval))),
        y0=[S0, I0, R0],
        t_eval=np.array(t_eval, dtype=float),
        rtol=1e-6,
        atol=1e-9,
    )
    if not sol.success:
        raise RuntimeError(sol.message)
    S, I, R = sol.y
    return S, I, R

# --- Demo simulation (starting guess) ---
t_eval = np.linspace(df["biweek"].min(), df["biweek"].max(), 200)

N = 50000      # TODO: adjust if you have a better population size estimate
gamma = 1.0    # per biweek (infectious period ~ 2 weeks)
beta = 3.0     # TODO: starting guess
I0 = 10.0      # TODO: starting guess

S, I, R = simulate_sir(t_eval, beta=beta, gamma=gamma, N=N, I0=I0)

# Sanity check: mass balance
mass_err = np.max(np.abs((S + I + R) - N))
print("Max |S+I+R-N| =", mass_err)

plt.figure()
plt.plot(t_eval, S, label="S")
plt.plot(t_eval, I, label="I")
plt.plot(t_eval, R, label="R")
plt.xlabel("Biweek")
plt.ylabel("Individuals")
plt.title("Closed SIR simulation (biweekly time)")
plt.legend()
plt.grid(True)
plt.show()


## 3. Feature-Based Estimation: Early Growth (Quick R₀)

We estimate early exponential growth using a semi-log regression:
- Choose an early window where cases grow roughly exponentially.
- Fit `log(cases)` vs time.

### ChatGPT Prompt (copy/paste)
```text
Persona:
You are an expert in infectious disease modeling.

Context:
- Community A measles outbreak
- Biweekly case counts
- Early outbreak phase shows exponential growth

Task:
Estimate early exponential growth by:
1) Selecting an early biweek window
2) Fitting log(measles) vs time
3) Plotting the semi-log fit
4) Converting slope to an approximate R0 assuming gamma = 1 per biweek

Constraints:
- Use numpy/scipy
- Explain assumptions clearly

Verification:
- Semi-log plot
- Printed slope and R0 estimate
```


In [ ]:
from numpy.linalg import lstsq

def early_growth_r0(df_comm, max_biweek=None, gamma=1.0, eps=1e-9):
    # Filter to early window (simple default)
    if max_biweek is None:
        max_biweek = df_comm["biweek"].min() + 6  # TODO: adjust (e.g., +8)
    d = df_comm[df_comm["biweek"] <= max_biweek].copy()
    d = d[d["measles"] > 0].copy()  # log requires > 0

    t = d["biweek"].values.astype(float)
    y = np.log(d["measles"].values.astype(float) + eps)

    # Linear regression y = a + b t
    X = np.column_stack([np.ones_like(t), t])
    coef, *_ = lstsq(X, y, rcond=None)
    a, b = coef

    # Simple heuristic: growth rate r ≈ (R0 - 1)*gamma  => R0 ≈ 1 + r/gamma
    r = b
    R0_hat = 1.0 + (r / gamma)
    return d, a, b, R0_hat

# Pick a community to start (A is common for demos)
comm = "A"
dfA = df[df["community"].astype(str) == comm].sort_values("biweek")

early_window_end = dfA["biweek"].min() + 8  # TODO: tweak window end
d_early, a_hat, b_hat, R0_hat = early_growth_r0(dfA, max_biweek=early_window_end, gamma=gamma)

print(f"Community {comm}")
print("Early window end biweek:", early_window_end)
print("Estimated growth slope b:", b_hat)
print("Approx R0:", R0_hat)

# Plot semi-log fit
plt.figure()
plt.scatter(d_early["biweek"], d_early["measles"], label="Observed (early window)")
t_line = np.linspace(d_early["biweek"].min(), d_early["biweek"].max(), 50)
y_line = np.exp(a_hat + b_hat * t_line)
plt.plot(t_line, y_line, label="Exponential fit")
plt.yscale("log")
plt.xlabel("Biweek")
plt.ylabel("Measles case counts (log scale)")
plt.title(f"Early growth (Community {comm})")
plt.grid(True)
plt.legend()
plt.show()

# Starting beta guess
beta0 = R0_hat * gamma
print("Suggested starting beta:", beta0)


## 4. Map Model Output to Observations (Incidence via ΔH)

Observed data are **biweekly case counts**, not the state `I(t)`.  
We model **incidence** using an accumulator:

- Add `H(t)` with **dH/dt = β S I / N**  
- Predicted biweekly counts ≈ **ΔH** over each biweek interval

### ChatGPT Prompt (copy/paste)
```text
Persona:
You are an expert epidemic modeler.

Context:
- Observed data are biweekly case counts (incidence)
- SIR model outputs states

Task:
Extend the SIR model by:
- Adding H(t) with dH/dt = beta*S*I/N
- Computing predicted biweekly cases as ΔH aligned to biweeks

Constraints:
- Keep code minimal and readable

Verification:
- Plot predicted ΔH
- Explain why ΔH matches count data
```


In [ ]:
def sirH_rhs(t, y, beta, gamma, N):
    """SIR + incidence accumulator H.
    y = [S, I, R, H]
    dH/dt = new infections per biweek
    """
    S, I, R, H = y
    incidence = beta * S * I / N
    dS = -incidence
    dI = incidence - gamma * I
    dR = gamma * I
    dH = incidence
    return [dS, dI, dR, dH]

def simulate_sir_incidence(biweeks, beta, gamma, N, I0, R0=0.0):
    """Simulate SIRH at specific biweek time points and return ΔH per interval."""
    S0 = N - I0 - R0
    t_eval = np.array(biweeks, dtype=float)
    sol = solve_ivp(
        fun=lambda t, y: sirH_rhs(t, y, beta=beta, gamma=gamma, N=N),
        t_span=(float(np.min(t_eval)), float(np.max(t_eval))),
        y0=[S0, I0, R0, 0.0],
        t_eval=t_eval,
        rtol=1e-6,
        atol=1e-9,
    )
    if not sol.success:
        raise RuntimeError(sol.message)
    S, I, R, H = sol.y
    dH = np.diff(H, prepend=H[0])  # first interval as 0
    return S, I, R, H, dH

# Example: predicted ΔH on the same biweek grid as the data for community A
biweeks_A = dfA["biweek"].values
S_A, I_A, R_A, H_A, dH_A = simulate_sir_incidence(
    biweeks=biweeks_A,
    beta=max(beta0, 1e-6),
    gamma=gamma,
    N=N,
    I0=I0,
)

plt.figure()
plt.plot(biweeks_A, dfA["measles"].values, marker="o", label="Observed cases")
plt.plot(biweeks_A, dH_A, marker="x", label="Predicted ΔH (incidence)")
plt.xlabel("Biweek")
plt.ylabel("Biweekly case counts")
plt.title("Observed vs predicted biweekly cases (Community A)")
plt.grid(True)
plt.legend()
plt.show()


## 5. Fit Parameters by Least Squares

We fit parameters by minimizing squared error between observed counts and predicted ΔH.

### ChatGPT Prompt (copy/paste)
```text
Persona:
You are an expert in numerical optimization for epidemic models.

Context:
- Observed data: biweekly measles case counts (incidence)
- Model prediction: ΔH from SIR + accumulator
- Time unit: biweeks

Task:
Fit beta (and optionally I0) by least squares:
- Use good initialization from early-growth estimates
- Use reasonable bounds
- Plot observed vs fitted cases

Constraints:
- Use scipy.optimize.least_squares
- Keep runtime suitable for a workshop

Verification:
- Overlay plot
- Print best-fit parameter values and SSE
```


In [ ]:
def fit_least_squares(df_comm, N, gamma=1.0, beta_init=3.0, I0_init=10.0):
    biweeks = df_comm["biweek"].values.astype(float)
    y_obs = df_comm["measles"].values.astype(float)

    def residuals(theta):
        # theta = [log_beta, log_I0]
        log_beta, log_I0 = theta
        beta = np.exp(log_beta)
        I0 = np.exp(log_I0)
        _, _, _, _, dH = simulate_sir_incidence(biweeks, beta=beta, gamma=gamma, N=N, I0=I0)
        return (dH - y_obs)

    x0 = np.log([max(beta_init, 1e-6), max(I0_init, 1e-6)])
    res = least_squares(
        residuals,
        x0=x0,
        bounds=(np.log([1e-6, 1e-6]), np.log([1e3, 1e6])),
        max_nfev=200,
    )
    beta_hat, I0_hat = np.exp(res.x)
    return beta_hat, I0_hat, res

beta_ls, I0_ls, res_ls = fit_least_squares(dfA, N=N, gamma=gamma, beta_init=beta0, I0_init=I0)
print("Least squares fit (Community A)")
print("beta_hat:", beta_ls)
print("I0_hat:", I0_ls)
print("SSE:", np.sum(res_ls.fun**2))

_, _, _, _, dH_fit = simulate_sir_incidence(dfA["biweek"].values, beta=beta_ls, gamma=gamma, N=N, I0=I0_ls)

plt.figure()
plt.plot(dfA["biweek"], dfA["measles"], marker="o", label="Observed")
plt.plot(dfA["biweek"], dH_fit, marker="x", label="Fitted (LS) ΔH")
plt.xlabel("Biweek")
plt.ylabel("Biweekly case counts")
plt.title("Least squares fit (Community A)")
plt.grid(True)
plt.legend()
plt.show()


## 6. Poisson Likelihood Inference (Counts)

For count data:
- \( y_t \sim \text{Poisson}(\mu_t) \)
- \( \mu_t = \rho \cdot \Delta H_t \)

where **ρ** is a reporting/ascertainment fraction.

### ChatGPT Prompt (copy/paste)
```text
Persona:
You are an expert in statistical inference for epidemic models.

Context:
- Observed data are biweekly count data
- Poisson errors are appropriate
- μ_t = rho * ΔH_t
- Time unit: biweeks

Task:
Implement MLE under Poisson likelihood:
- Use parameter transforms (log for beta, logit for rho, log for I0)
- Optimize negative log-likelihood
- Plot observed vs expected counts

Constraints:
- Use scipy.optimize.minimize
- Keep runtime suitable for a workshop

Verification:
- Overlay plot of observed vs expected
- Print MLE estimates and log-likelihood
```


In [ ]:
from scipy.special import gammaln

def logit(p):
    p = np.clip(p, 1e-9, 1-1e-9)
    return np.log(p/(1-p))

def inv_logit(x):
    return 1/(1+np.exp(-x))

def poisson_nll(y, mu):
    mu = np.clip(mu, 1e-12, None)
    return np.sum(mu - y*np.log(mu) + gammaln(y + 1))

def fit_poisson_mle(df_comm, N, gamma=1.0, beta_init=3.0, I0_init=10.0, rho_init=0.2):
    biweeks = df_comm["biweek"].values.astype(float)
    y_obs = df_comm["measles"].values.astype(float)

    def objective(theta):
        # theta = [log_beta, log_I0, logit_rho]
        log_beta, log_I0, logit_rho = theta
        beta = np.exp(log_beta)
        I0 = np.exp(log_I0)
        rho = inv_logit(logit_rho)

        _, _, _, _, dH = simulate_sir_incidence(biweeks, beta=beta, gamma=gamma, N=N, I0=I0)
        mu = rho * dH
        return poisson_nll(y_obs, mu)

    x0 = np.array([np.log(max(beta_init, 1e-6)), np.log(max(I0_init, 1e-6)), logit(rho_init)], dtype=float)

    res = minimize(
        objective,
        x0=x0,
        method="Nelder-Mead",
        options=dict(maxiter=1500, xatol=1e-6, fatol=1e-6),
    )

    log_beta, log_I0, logit_rho = res.x
    beta_hat = np.exp(log_beta)
    I0_hat = np.exp(log_I0)
    rho_hat = inv_logit(logit_rho)

    nll = objective(res.x)
    return beta_hat, I0_hat, rho_hat, nll, res

beta_mle, I0_mle, rho_mle, nll_mle, res_mle = fit_poisson_mle(
    dfA, N=N, gamma=gamma, beta_init=beta_ls, I0_init=I0_ls, rho_init=0.2
)

print("Poisson MLE (Community A)")
print("beta_hat:", beta_mle)
print("I0_hat:", I0_mle)
print("rho_hat:", rho_mle)
print("NLL:", nll_mle)
print("Converged:", res_mle.success)

_, _, _, _, dH_mle = simulate_sir_incidence(dfA["biweek"].values, beta=beta_mle, gamma=gamma, N=N, I0=I0_mle)
mu_mle = rho_mle * dH_mle

plt.figure()
plt.plot(dfA["biweek"], dfA["measles"], marker="o", label="Observed")
plt.plot(dfA["biweek"], mu_mle, marker="x", label="Expected (Poisson MLE)")
plt.xlabel("Biweek")
plt.ylabel("Biweekly case counts")
plt.title("Poisson likelihood fit (Community A)")
plt.grid(True)
plt.legend()
plt.show()


## 7. Quick Uncertainty: Profile Likelihood for β (Optional)

### ChatGPT Prompt (copy/paste)
```text
Persona:
You are an expert in likelihood-based inference.

Context:
- We already have a Poisson likelihood fit
- We want uncertainty for beta

Task:
Create a profile likelihood for beta:
1) Fix beta over a grid
2) Optimize I0 and rho for each beta
3) Plot profile NLL(beta)

Constraints:
- Keep runtime reasonable for a workshop

Verification:
- Profile plot
- Explain how to interpret the uncertainty
```


In [ ]:
def profile_beta(df_comm, N, gamma, beta_grid, I0_init, rho_init):
    biweeks = df_comm["biweek"].values.astype(float)
    y_obs = df_comm["measles"].values.astype(float)

    def objective_I0_rho(theta, beta_fixed):
        # theta = [log_I0, logit_rho]
        log_I0, logit_rho = theta
        I0 = np.exp(log_I0)
        rho = inv_logit(logit_rho)
        _, _, _, _, dH = simulate_sir_incidence(biweeks, beta=beta_fixed, gamma=gamma, N=N, I0=I0)
        mu = rho * dH
        return poisson_nll(y_obs, mu)

    nlls = []
    for b in beta_grid:
        x0 = np.array([np.log(max(I0_init, 1e-6)), logit(rho_init)], dtype=float)
        res = minimize(
            lambda th: objective_I0_rho(th, beta_fixed=b),
            x0=x0,
            method="Nelder-Mead",
            options=dict(maxiter=800, xatol=1e-6, fatol=1e-6),
        )
        nlls.append(objective_I0_rho(res.x, beta_fixed=b))
    return np.array(nlls)

beta_grid = np.linspace(max(1e-6, 0.5*beta_mle), 1.5*beta_mle, 35)
nll_prof = profile_beta(dfA, N=N, gamma=gamma, beta_grid=beta_grid, I0_init=I0_mle, rho_init=rho_mle)

plt.figure()
plt.plot(beta_grid, nll_prof, marker="o")
plt.xlabel("beta")
plt.ylabel("Negative log-likelihood (profiled)")
plt.title("Profile likelihood for beta (Community A)")
plt.grid(True)
plt.show()

print("Min NLL:", nll_prof.min(), "at beta ≈", beta_grid[np.argmin(nll_prof)])


## 8. Generate a Markdown Summary (Hackathon Report Cell)

### ChatGPT Prompt (copy/paste)
```text
Persona:
You are preparing a short scientific report for hackathon participants.

Context:
- We modeled biweekly measles case counts in Niamey (community A)
- We used a closed SIR model with gamma ≈ 1 per biweek
- We estimated beta and rho via Poisson likelihood

Task:
Write a Markdown summary including:
- Model setup and key assumptions
- Parameter estimates (beta, gamma, rho) and R0
- Observed vs fitted comparison (brief interpretation)
- Key limitations
- Next steps (SEIR, time-varying beta, stochasticity)

Constraints:
- Scientific but accessible
- Use headers + bullet points
```


In [ ]:
# TODO: Paste ChatGPT-generated Markdown summary below this line (or create a new Markdown cell).
summary = f'''
## Hackathon Summary (Community A)

**Data:** Biweekly measles case counts (Niamey, Niger), Community A  
**Model:** Closed SIR with incidence accumulator (ΔH)  
**Time unit:** biweeks  
**Assumed recovery rate:** γ = {gamma:.3g} per biweek

### Estimated parameters
- β (Poisson MLE): **{beta_mle:.4g}**
- ρ (Poisson MLE): **{rho_mle:.4g}**
- Approx R₀ = β/γ: **{(beta_mle/gamma):.4g}**

### Notes
- ΔH was used to map model infections to observed case counts.
- Fit quality should be assessed visually (Observed vs Expected plot) and via NLL.

### Next steps
- Fit communities B and C
- Compare Poisson vs Negative Binomial (overdispersion)
- Try SEIR or time-varying β(t)
'''
print(summary)
